# Clase 2
En esta clase vamos a poner en practica los conocimiento vistos en la primera clase.

Vamos a utilizar un dataset bastante conocido: Titanic, que muestra datos de los pasajeros del crucero. El objetivo es entrenar un clasificador binario que, a partir de los datos de los pasajeros, clasifique correctamente su supervivencia.

Comencemos por descargarlo en la siguiente celda:

In [14]:
! wget https://eva.fing.edu.uy/pluginfile.php/255092/mod_folder/content/0/titanic.txt

--2023-09-27 14:20:56--  https://eva.fing.edu.uy/pluginfile.php/255092/mod_folder/content/0/titanic.txt
Resolving eva.fing.edu.uy (eva.fing.edu.uy)... 164.73.32.9
Connecting to eva.fing.edu.uy (eva.fing.edu.uy)|164.73.32.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116946 (114K) [text/plain]
Saving to: ‘titanic.txt.1’

titanic.txt.1       100%[===================>] 114.21K   223KB/s    in 0.5s    

2023-09-27 14:20:58 (223 KB/s) - ‘titanic.txt.1’ saved [116946/116946]



La siguiente celda es para verificar que la descarga fue exitosa. En caso de tener problemas, pueden hacerlo manualmente desde el EVA del curso, en el material de la clase 2.

In [15]:
import os
assert os.path.isfile("titanic.txt"), "No se descargo el archivo! Verificar la ejecucion de la celda anterior."

Luego vamos a importar algunas librerias genericas.

_Nota: Siempre es buena idea importar las librerías genéricas al principio._

In [16]:
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt

RANDOM_STATE=0

## Importación y procesamiento de datos
En todo proyecto de aprendizaje automático es fundamental manejar el conjunto de datos.
Es importante tener una noción del conjunto de datos, para saber, entre otras cosas:
* Qué atributos son numéricos, y cuáles son categóricos.
* Si hace falta normalizar los atributos (depende del algoritmo a utilizar también)
* La presencia de atributos faltantes.

### Importación
Primero importamos los datos.  
Al observar el archivo, pueden identificar columnas e instancias que no sean necesarias?

In [17]:
import os
import pandas as pd

assert os.path.isfile('titanic.txt'), 'No se encontró el archivo titanic.txt; asegurate de haberlo cargado'

# leemos el dataset utilizando Pandas
data = pd.read_csv('titanic.txt')
# eliminamos la columna row.names que solo tiene el nuero de fila
# tambien vamos a eliminar los atributos 'name' y 'home.dest'
# ya que contienen texto libre, y aun no hemos visto como tratar con ellos
data.drop(['row.names', 'name', 'home.dest'], axis=1, inplace=True)

print('Mostramos para cada columna, el porcentaje de datos faltantes:\n')
print(data.isnull().mean()*100)

Mostramos para cada columna, el porcentaje de datos faltantes:

pclass       0.000000
survived     0.000000
age         51.789794
embarked    37.471439
room        94.135567
ticket      94.744859
boat        73.571973
sex          0.000000
dtype: float64


In [18]:
# === Su codigo empieza acá ===
# Modificar la lista columns de manera que contenga solo aquellos atributos
# que querramos a usar. Notar que actualmente tiene todos los atributos
# borren aquellos que crean inutiles

columns = ['pclass', 'embarked', 'sex', 'age'] # elimino boat, room, ticket

# === Su codigo termina acá ===

# Nos quedamos con los datos como numpy array
X = data[columns].values
y = data['survived'].values

print('X tiene forma', X.shape)
print('y tiene forma', y.shape)

X tiene forma (1313, 4)
y tiene forma (1313,)


En la siguiente celda, interesa contar cuantos casos hay en cada clase, para elegir una metrica apropiada:

_Pista: hay varias formas de hacerlo, por ejemplo, pueden usar la función [`np.unique`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html)_

In [19]:
# === Su código empieza acá ===
# value_counts()
print ('Valores únicos de columna "survived" - value_counts()')
print(f"{data['survived'].value_counts()}\n")

# np.unique
print ('Valores únicos de columna "survived" - np.unique')
for cat,count in zip(np.unique(y, return_counts = True)[0], np.unique(y, return_counts = True)[1]):
    print (f'Survived: {cat} - Cantidad = {count}')
# === Su código termina acá ===

Valores únicos de columna "survived" - value_counts()
0    864
1    449
Name: survived, dtype: int64

Valores únicos de columna "survived" - np.unique
Survived: 0 - Cantidad = 864
Survived: 1 - Cantidad = 449


Vamos a visualizar algunos ejemplos al azar, utilizando la funcion `show_some_samples`:

In [20]:
def show_some_samples(X, y, columns=columns, n_samples=3, seed=None):
  """
  show random instances from X with its label in y.
  X: dataset to sample, as a numpy array of shape (n_samples, n_features)
  y: labels, as a numpy array of shape (n_samples,)
  columns: list of string with the name of each column, so len(columns) == n_features
  n_samples: number of samples to show
  seed: seed to set before choosing examples
  """
  if seed is not None:
    np.random.seed(seed=seed)

  idx = np.random.choice(len(X), n_samples)

  for i, (x, t) in enumerate(zip(X[idx], y[idx])):
    print(f'==== idx {idx[i]:6d} :: target = {t} ====')
    for feat_name, feat_value in zip(columns, x):
      print(f'\t{feat_name}: {feat_value}')

# aca esta la invocacion, no tienen que cambiar nada
show_some_samples(X, y)

==== idx   1015 :: target = 0 ====
	pclass: 3rd
	embarked: nan
	sex: male
	age: nan
==== idx   1208 :: target = 0 ====
	pclass: 3rd
	embarked: nan
	sex: male
	age: nan
==== idx   1280 :: target = 0 ====
	pclass: 3rd
	embarked: nan
	sex: male
	age: nan


# Pregunta A

Qué debería hacer primero:  


1.   Rellenar los datos faltantes con la politica elegida (por ejemplo, el más frecuente)
2.   Partir el dataset en entrenamiento y test

**Respuesta:**
En primer lugar se debería separar el dataset en entrenamiento y test, y luego rellenar los datos faltantes de ambos sets con el valor más frecuente del set de entrenamiento. Si se rellenaran los datos faltantes utilizando todo el dataset, se estaría realizando el preprocesamiento del set de entrenamiento utilizando datos que van a formar parte del test set. Esto podría llevar a la sobreestimación de la performance del modelo, ya que se estaría evaluando el modelo con datos que fueron utilizados para la preparación de los datos de entrenamiento. En cambio, realizar la partición del dataset antes de hacer la imputación permite emular una situación lo más cercana posible a una situación real y asegura la consistencia entre el set de entrenamiento y de test, ya que se está imputando el valor más frecuente de los datos de entrenamiento, y estos datos serán los únicos disponibles al momento de realizar una predicción sobre datos nuevos cuando el modelo esté en producción.   

*Nota* Ajuste el orden de las siguientes celdas de acuerdo a lo que crea más conveniente.


# 1: Separar train y test
En la siguiente celda, utilizar la funcion `sklearn.model_selection.train_test_split` para separar el dataset en entrenamiento y test. Vamos a tomar un 30% para test, y utilizar como semilla la constante `RANDOM_STATE`:

In [21]:
from sklearn.model_selection import train_test_split

# === Su código empieza acá ===
# X_train, X_test, y_train, y_test = # completar la invocación
x = data[columns]
y = data['survived']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = RANDOM_STATE)
# === Su código termina acá ===

# El siguiente codigo es un chequeo automatico de que todo va bien
# Si no salta ningun error, es porque esta todo Ok
assert len(X_train) == len(y_train), f'X_train e y_train deberian tener la misma cantidad de elementos: {len(X_train)} != {len(y_train)}'
assert len(X_test) == len(y_test), f'X_test e y_test deberian tener la misma cantidad de elementos: {len(X_test)} != {len(y_test)}'
assert X_train.shape[1] == X_test.shape[1], f'X_train y X_test deberian tener los mismos atributos: {X_test.shape[1]} != {X_test.shape[1]}'
assert len(X) * 0.28 < len(X_test) < len(X) * 0.32, 'Verificar que el test sea 30%'

# 2: Imputar datos faltantes
Utilizar la clase `sklearn.impute.SimpleImputer` para rellenar los atributos faltantes.

Probar la estrategia `mean` y `most_frequent`.

# Pregunta B

**Cuál crees que es más adecuada? Justifique**

**Respuesta:**
Dado que, de las cuatro columnas que serán utilizadas, tres de ellas corresponden a variables categóricas y una corresponde a una variable numérica, es más adecuado realizar la imputación con el valor más frecuente (ya que no es posible calcular el promedio de valores de una variable categórica)

In [22]:
from sklearn.impute import SimpleImputer

# === Su código empieza acá ===
# definir el imputer y entrenarlo
imputer = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
imputer.fit(X_train)
# === Su código termina acá ===

X_train_fill = imputer.transform(X_train)
X_test_fill = imputer.transform(X_test)

In [23]:
# Chequeo que ya no existan valores nulos

# Antes de imputación
print(f'Valores nulos antes de imputación')
print(f'Entrenamiento:\n{X_train.isna().sum()}\n')
print(f'Testeo:\n{X_test.isna().sum()}\n')

# Luego de imputación
print(f'Valores nulos luego de imputación')
print(f'Entrenamiento: {(X_train_fill == np.nan).sum()}')
print(f'Testeo: {(X_test_fill == np.nan).sum()}')

Valores nulos antes de imputación
Entrenamiento:
pclass        0
embarked    334
sex           0
age         471
dtype: int64

Testeo:
pclass        0
embarked    158
sex           0
age         209
dtype: int64

Valores nulos luego de imputación
Entrenamiento: 0
Testeo: 0


En la siguiente celda vamos a ver como son los contenidos de cada atributo: cuantos hay de cada tipo.

El objetivo de su ejecución es ayudarnos a decidir cuáles son y cómo vamos a codificar los atributos categóricos.

In [24]:
# Iteramos sobre cada una de las columnas
for idx, clm in enumerate(columns):
  print(f'==={idx}: {clm}===')
  # Para cada columna, contamos la cantidad de valores unicos que hay
  unq, cnt = np.unique(X_train_fill[:, idx], return_counts=True)
  for u, c in zip(unq, cnt):
    # mostramos cada valor unico, con la cantidad que hay,
    # y qué porcentaje representa del dataset
    print(f'\t{u}: {c} - {100*c/cnt.sum():5.2f} %')

===0: pclass===
	1st: 231 - 25.14 %
	2nd: 189 - 20.57 %
	3rd: 499 - 54.30 %
===1: embarked===
	Cherbourg: 150 - 16.32 %
	Queenstown: 30 -  3.26 %
	Southampton: 739 - 80.41 %
===2: sex===
	female: 323 - 35.15 %
	male: 596 - 64.85 %
===3: age===
	0.1667: 1 -  0.11 %
	0.3333: 1 -  0.11 %
	0.8333: 2 -  0.22 %
	0.9167: 1 -  0.11 %
	1.0: 1 -  0.11 %
	2.0: 3 -  0.33 %
	3.0: 4 -  0.44 %
	4.0: 4 -  0.44 %
	5.0: 1 -  0.11 %
	6.0: 4 -  0.44 %
	7.0: 1 -  0.11 %
	8.0: 3 -  0.33 %
	9.0: 7 -  0.76 %
	10.0: 1 -  0.11 %
	11.0: 1 -  0.11 %
	12.0: 2 -  0.22 %
	13.0: 3 -  0.33 %
	14.0: 2 -  0.22 %
	15.0: 3 -  0.33 %
	16.0: 9 -  0.98 %
	17.0: 6 -  0.65 %
	18.0: 14 -  1.52 %
	19.0: 13 -  1.41 %
	20.0: 9 -  0.98 %
	21.0: 17 -  1.85 %
	22.0: 15 -  1.63 %
	23.0: 16 -  1.74 %
	24.0: 12 -  1.31 %
	25.0: 13 -  1.41 %
	26.0: 490 - 53.32 %
	27.0: 13 -  1.41 %
	28.0: 15 -  1.63 %
	29.0: 9 -  0.98 %
	30.0: 18 -  1.96 %
	31.0: 8 -  0.87 %
	32.0: 14 -  1.52 %
	33.0: 10 -  1.09 %
	34.0: 8 -  0.87 %
	35.0: 13 -  1.41 %
	

# 3: Codificar atributos categóricos
El siguente paso va a ser codificar los atributos categóricos.

En clase, mencionamos principalmente dos estrategias: `sklearn.preprocessing.OrdinalEncoder` y `sklearn.preprocessing.OneHotEncoder`. Utilice la (o las) que crea más conveniente.

**PISTA**

Hasta el momento, tenemos un maximo de 8 atributos. Sin embargo, **no todos ellos son categoricos**, por lo que en realidad solo necesito transformar alguno de ellos.

Ademas, para cada uno, podria necesitar una codificacion distinta.

Para esto, nos vamos a ayudar del transformador `sklearn.compose.ColumnTransformer`, que permite aplicar un transformador diferente a cada atributo (columna), y cuyo funcionamiento es el siguiente:

```python
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

transformers = [
        # ('nombre_arbitrario', Transformador, [indices])
        ("trnf1", OrdinalEncoder(), [0]),
        ("trnf2", OneHotEncoder(), [1, 2]),
        ("scaler", MinMaxScaler(), [3])
     ]

ct = ColumnTransformer(transformers, remainder='passthrough')

ct.fit(X)
X_trans = ct.transform(X)
```

El `ColumnTransformer` recibe una lista de transformadores a aplicar, indicando a qué columna aplicarlo, y se debe especificar qué hacer con el resto de las columnas. En el ejemplo, `reminder='passthrough'` quiere decir que los valores se pasan de largo sin ninguna modificación.

In [25]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

# === Su código empieza acá ===
# sugerencia: verificar el shape antes y despues de las transformaciones
# para asegurar que esta todo coherente
# definir el column transformer (ct) y entrenarlo

transformers = [
        ("ord", OrdinalEncoder(), [0]),
        ("one_hot", OneHotEncoder(), [1, 2]),
        ("scaler", MinMaxScaler(), [3])
     ]

ct = ColumnTransformer(transformers, remainder = 'passthrough')

# Entrenamiento
ct.fit(X_train_fill)

# === Su código termina acá ===
X_train_fill_num = ct.transform(X_train_fill)

Verificar el shape, que tenga sentido

In [26]:
print (f'Train set shape before transformation: {X_train_fill.shape}')
print (f'Train set shape after transformation: {X_train_fill_num.shape}')
print ('-'*80)
print (f'Variables luego de transformación:')

for i,var in enumerate (ct.get_feature_names_out()):
    print (f'Var {i}: \t {var} \t type: {type(X_train_fill_num[:,i][0]).__name__}')

print ('-'*80)

Train set shape before transformation: (919, 4)
Train set shape after transformation: (919, 7)
--------------------------------------------------------------------------------
Variables luego de transformación:
Var 0: 	 ord__x0 	 type: float64
Var 1: 	 one_hot__x1_Cherbourg 	 type: float64
Var 2: 	 one_hot__x1_Queenstown 	 type: float64
Var 3: 	 one_hot__x1_Southampton 	 type: float64
Var 4: 	 one_hot__x2_female 	 type: float64
Var 5: 	 one_hot__x2_male 	 type: float64
Var 6: 	 scaler__x3 	 type: float64
--------------------------------------------------------------------------------


# Pregunta C
Cuantas columnas tiene, por qué y qué representa cada una?

_Nota: no es estrictamente necesario saber cada columna con qué se corresponde (es decir: que hay en la columna 0? que hay en la columna 1? ... no es necesario responder a ese nivel)._

_Se espera que si en este punto tienen, por ejemplo, 13 columnas, explique cuáles son y cómo llegaste a ellas._

**Respuesta:**
Se tienen 7 columnas. Dado que se usó OneHotEncoder para las variables "embarked" y "sex", y dichas variables tienen 3 y 2 valores únicos respectivamente, se generan 3 columnas para cada valor de "embarked" y 2 para cada valor de "sex". Luego, las 2 columnas restantes corresponden a la codificación ordinal de "pclass" y a la normalización de "age".


# 4: Seleccionar atributos
Utilizar alguna de las estrategias vistas en clase para quedarnos con 5 atributos.

In [27]:
from sklearn.feature_selection import RFE, SelectKBest, chi2, SequentialFeatureSelector, f_classif
from sklearn.tree import DecisionTreeClassifier

# === Su código empieza acá ===
# definir el feature selector (fs) y entrenarlo

print (f'Antes de selección de atributos: {X_train_fill_num.shape}')
fs = SelectKBest(f_classif, k = 5)
fs.fit(X_train_fill_num, y_train)

# === Su código termina acá ===
X_train_fill_num_selected = fs.transform(X_train_fill_num)
print (f'Luego de selección de atributos: {X_train_fill_num_selected.shape}')

Antes de selección de atributos: (919, 7)
Luego de selección de atributos: (919, 5)


# 5: Entrenar un clasificador
En el siguiente paso, vamos a entrenar un clasificador [`sklearn.tree.DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) con los datos siguiendo todas las transformaciones que seguimos hasta acá.

Utilizar `sklearn.model_selection.GridSearchCV` o `sklearn.model_selection.RandomizedSearchCV` para seleccionar los mejores parametros.

Utilizar validacion cruzada con 10 particiones. Utilizar una metrica acorde al problema.

# Pregunta D
 - Cuál es la **menor** cantidad de particiones que puedo usar?
 - Cuál es la **mayor** cantidad de particiones que puedo usar?
 - Qué pasa en cada uno de estos extremos?
 - Qué métrica vas a usar? **Justifique brevemente**



#### **Respuestas**

1. La **menor** cantidad de particiones que se puede usar es 1, en cuyo caso no se estaría utilizando validación cruzada propiamente dicho. Utilizar una única partición implicaría partir el dataset de entrenamiento una única vez, obteniendo un set de entrenamiento con el cual se entrenará el clasificador y un set de validación con el cual se lo evaluará. Dado que se tiene una única partición, no habrá más etapas de entrenamiento ni se promediará ningún resultado, ya que el único resultado de performance del modelo será el obtenido al evaluarlo con el set de validación. La mayor desventaja de este método es la variabilidad de los resultados, ya que distintas posibles particiones pueden llevar a resultados distintos.

2. La **mayor** cantidad de particiones que se puede usar es **n**, siendo **n** la cantidad de observaciones del set de entrenamiento (en este caso, X_train). En este caso, el cual es denominado Leave One Out Cross Validation (LOOCV), se entrenan **n** modelos distintos. Cada modelo es entrenado con **n-1** datos y es evaluado con el dato restante. Luego, se promedian los **n** resultados para obtener el resultado final. Este método tiene muy poco sesgo debido a que se utilizan todos los datos (excepto  uno) para entrenar el modelo, pero tiene mucha varianza ya que todos los modelos construidos están muy correlacionados. Asimismo, el costo computacional puede ser alto debido a que se debe entrenar **n** modelos distintos.

3. Como se vio más arriba, las clases están desbalanceadas, ya que 449 pasajeros sobrevivieron y 864 no sobrevivieron. En virtud de esto, se opta por no utilizar la métrica de Accuracy, ya que la misma puede tomar valores altos aún cuando la clase minoritaria tenga un error de clasificación relativamente alto. Asimismo, a priori no parece que el costo de clasificar erróneamente alguna de las clases sea mayor que el costo de clasificar erróneamente la otra clase, es decir, el costo de los falsos positivos y falsos negativos es considerado como igual. Luego, dado que no se busca penalizar ningún tipo de error sobre el otro, se opta por la utilización de la métrica **f1-score** para la evaluación del modelo. Esta métrica es igual a la media armónica de Recall y Precision, y representa una ponderación de ambas, lo que permite tener una evaluación indirecta de los errores de clasificación de ambas clases. Si, por ejemplo, el modelo clasificara la gran mayoría de los datos en una única clase, Recall o Precision podrían tener un valor muy cercano a 1 de manera individual, pero no de manera simultánea, lo que se verá reflejado en el f1-score. Cabe destacar, igualmente, que se utiliza esta métrica para la comparación de modelos y elección de hiperparámetros, pero se utilizan las cuatro ya mencionadas para la evaluación del modelo final elegido.


In [28]:
# Definición paramétrica de métrica a utilizar
scoring = 'f1'

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# === Su código empieza acá ===
# definir la grid search o random search  (grid) y entrenarlo
param_distributions = {'criterion': ['gini', 'entropy'],
                    'max_depth': [None, 2,3,4,5],
                    'max_features': [5, 'sqrt']
                    }

classif_tree = DecisionTreeClassifier(random_state = RANDOM_STATE)
grid = GridSearchCV(classif_tree, param_distributions, cv = 10, scoring = scoring)

# === Su código termina acá ===
grid.fit(X_train_fill_num_selected, y_train)

print(grid.best_params_)
grid.best_score_

{'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt'}


0.6728221481085306

# 6: pipeline
Compactar todos los pasos ejecutados hasta ahora en un mismo Pipeline.

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.base import BaseEstimator, TransformerMixin


# === Su código empieza acá ===
# definir la pipeline (pipe) y complete con la metrica seleccionada
transformers = [
        ("ord", OrdinalEncoder(), [0]),
        ("one_hot", OneHotEncoder(), [1, 2]),
        ("scaler", MinMaxScaler(), [3])
        ]

pipe = Pipeline(
    [('imputer', SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')),
    ('transformer', ColumnTransformer(transformers = transformers, remainder = 'passthrough')),
    ('selector', SelectKBest(f_classif, k = 5)),
    ('classifier', DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 'sqrt', random_state = RANDOM_STATE))
    ]
)

result = cross_validate(pipe, X_train, y_train, cv=10, scoring=scoring)
# === Su código termina acá ===


score_mean = result['test_score'].mean()
score_std = result['test_score'].std()

print(f'score obtenido: {score_mean:.3f} ± {score_std:.3f} %')

score obtenido: 0.673 ± 0.027 %


# 7: Obtener el mejor clasificador posible

Ahora que todos los pasos estan dentro de un pipeline, podemos re ver las desiciones tomadas en cada paso para obtener el mejor clasificador posible.


In [31]:
# Definición de algoritmos de transformación para el step "transformers"
# Se evalúan opciones con y sin normalización de la columna de age, así como de
# codificación nominal y ordinal para la variable de pclass.

from sklearn.preprocessing import StandardScaler

transformers_1 = [
        ("ord", OrdinalEncoder(), [0]),
        ("one_hot", OneHotEncoder(), [1, 2]),
        ("scaler", MinMaxScaler(), [3])
        ]

transformers_2 = [
        ("ord", OrdinalEncoder(), [0]),
        ("one_hot", OneHotEncoder(), [1, 2])
        ]

transformers_3 = [
        ("one_hot", OneHotEncoder(), [0, 1, 2]),
        ("scaler", MinMaxScaler(), [3])
        ]

transformers_4 = [
        ("one_hot", OneHotEncoder(), [0, 1, 2])
        ]

# Listado para pasar como parámetros en grid search
transformers = [transformers_1,transformers_2, transformers_3, transformers_4]

In [32]:
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE, SelectKBest, chi2, SequentialFeatureSelector, f_classif, mutual_info_classif

# === Su código empieza acá ===
# de ser necesario, agreguen más celdas
# de ser necesario, importen otros modulos
# Si quieren, aprovechen a rever todas las decisiones tomadas hasta acá

pipe = Pipeline(
    [('imputer', SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')),
    ('transformer', ColumnTransformer(transformers = transformers_1, remainder = 'passthrough')),
    ('selector', SelectKBest()),
    ('classifier', DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 5))
    ])

params = [
            {# KNN
        'transformer__transformers': transformers,
        'selector__score_func':[f_classif, mutual_info_classif, chi2],
        'selector__k': [4,5,6, 'all'],
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': [5,10,15,20,25,30],
        'classifier__weights': ['uniform', 'distance']
        },

            # Random Forest
        {'transformer__transformers': transformers,
        'selector__score_func':[f_classif, mutual_info_classif, chi2],
        'selector__k': [4,5,6, 'all'],
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [10, 50, 100, 200],
        'classifier__criterion': ['gini', 'entropy', 'log_loss'],
        'classifier__max_features': ['sqrt', 'log2', None]
        },

            # Support Vector Classifier
        {'transformer__transformers': transformers,
        'selector__score_func':[f_classif, mutual_info_classif, chi2],
        'selector__k': [4,5,6, 'all'],
        'classifier': [SVC()],
        'classifier__C': [1,2,3,4,5,10,20],
        'classifier__kernel': ['poly', 'rbf'],
        'classifier__class_weight': ['balanced', None]
        },

            # Logistic Regression
        {'transformer__transformers': transformers,
        'selector__score_func':[f_classif, mutual_info_classif, chi2],
        'selector__k': [4,5,6, 'all'],
        'classifier': [LogisticRegression()],
        'classifier__penalty': ['l2',None],
        'classifier__class_weight': ['balanced', None]
        },

            # Gradient Boosting Classifier
        {'transformer__transformers': transformers,
        'selector__score_func':[f_classif, mutual_info_classif, chi2],
        'selector__k': [4,5,6, 'all'],
        'classifier': [GradientBoostingClassifier()],
        'classifier__loss': ['exponential', 'log_loss'],
        'classifier__max_features': ['sqrt', 'log2', None]
        }
    ]

# En primer lugar, se realiza una búsqueda de hiperparámetros con RandomizedSearchCV(), de manera de identificar el clasificador con mejor
# métrica. Se fija el número de iteraciones en 100 de manera de probar con una cantidad razonable de combinaciones para cada algoritmo sin aumentar demasiado el costo.
# Luego, se realizará una búsqueda exhaustiva con GridSearchCV buscando optimizar los parámetros de ese algoritmo.

pipe_grid = RandomizedSearchCV(pipe, params, cv = 10, scoring = scoring, error_score = 'raise', n_iter = 100, random_state = RANDOM_STATE)
pipe_grid.fit(X_train, y_train)

print(pipe_grid.best_params_)
pipe_grid.best_score_


{'transformer__transformers': [('ord', OrdinalEncoder(), [0]), ('one_hot', OneHotEncoder(), [1, 2])], 'selector__score_func': <function chi2 at 0x78a1475653f0>, 'selector__k': 6, 'classifier__kernel': 'rbf', 'classifier__class_weight': None, 'classifier__C': 5, 'classifier': SVC(C=5)}


0.6912228135956948

In [33]:
# Support Vector Classifier

params_opt = {
        'transformer__transformers': [transformers_1, transformers_2],
        'selector__score_func':[chi2],
        'selector__k': [6, 'all'],
        'classifier': [SVC()],
        'classifier__C': [1,2,3,4,5,10,20],
        'classifier__kernel': ['poly', 'rbf'],
        'classifier__class_weight': ['balanced', None]
}

pipe_grid_opt = GridSearchCV(pipe, params_opt, cv = 10, scoring = scoring, error_score = 'raise')

pipe_grid_opt.fit(X_train, y_train)

print(pipe_grid_opt.best_params_)
pipe_grid_opt.best_score_

{'classifier': SVC(C=10), 'classifier__C': 10, 'classifier__class_weight': None, 'classifier__kernel': 'rbf', 'selector__k': 6, 'selector__score_func': <function chi2 at 0x78a1475653f0>, 'transformer__transformers': [('ord', OrdinalEncoder(), [0]), ('one_hot', OneHotEncoder(), [1, 2])]}


0.7025605725973374

# 8: evaluacion


Una vez encontrado este clasificador, evaluarlo sobre el dataset de test con la funcion `sklearn.metrics.classification_report`

In [34]:
from sklearn.metrics import classification_report

# === Su código empieza acá ===
# Utilizar el mejor modelo encontrado para clasificar X_test
# Asegurate de que este entrenado con los datos correctos

y_pred = pipe_grid_opt.predict(X_test)
# === Su código termina acá ===
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.81      0.84       264
           1       0.66      0.75      0.70       130

    accuracy                           0.79       394
   macro avg       0.76      0.78      0.77       394
weighted avg       0.80      0.79      0.79       394



# Pregunta E
Suponiendo que el resultado de la celda anterior es este:
```python
              precision    recall  f1-score   support

           0       0.84      0.81      0.82       264
           1       0.64      0.68      0.66       130

    accuracy                           0.77       394
   macro avg       0.74      0.75      0.74       394
weighted avg       0.77      0.77      0.77       394
```

Interprete con sus palabras, para una persona no tecnica, la implicancia de obtener:

- precision = 0.84 para la clase 0
- recall = 0.68 para la clase 1

**Respuesta**
1. Un valor de precision de 0.84 para la clase 0 implica que, de todos los pasajeros del conjunto de testeo para los cuales se predijo que el pasajero no sobrevivió, se predijo correctamente al 84%.
2. Un valor de recall de 0.68 para la clase 1 implica que, del total de pasajeros del conjunto de testeo que efectivamente sobrevivieron, se clasificó correctamente al 68%.